<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=7314cec42557546b62bf646ff5b70190ced0aa95e79efc83fd02a76ec96640f6
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1


    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-19 11:19:16
-------------------
qualified stocks: 88
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.60 K
Current:  1.45 C
-------------------
Today PnL: -29.84 K (-0.21%)
Current PnL: -21.29 L (-13.95%)
CY Booked + Current PnL: -7.33 L (-4.81%)
-------------------
Total profit:  1.74 L
Total loss:  -23.03 L
-------------------
Total Booked + Current PnL: 19.65 L (15.64%)
Total Booked PnL: 40.94 L (32.59%)
Curr Year Booked PnL: 13.96 L (9.66%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.35 C
Est FTT PnL: 90.64 L (62.7%)
Deployed:  1.26 C
Current:  1.45 C
CAGR/XIRR %: 8.08%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,146.65,79.0,M-SC,1.89,161777.0,-12782.0,12813.0,-1.61,-7.32,7.92,0.02,234.0,-1.00,1.12,40.72,OX40N,NTT,PAINTS
79,TTKPRESTIG,770.00,100.85,50.0,M-SC,3.72,86526.0,-14251.0,14346.0,0.09,-14.14,16.58,0.09,253.0,-0.99,0.60,13.68,OX40N,NTT,DURABLES
2,ABBOTINDIA,35195.00,-14.73,50.0,X-MC,3.84,89010.0,-1668.0,16574.0,0.02,-1.84,18.62,16.44,63.0,-0.10,0.62,17.96,X40,ATH,PHARMA
84,VOLTAS,1530.00,-1.35,55.0,X-MC,3.72,209625.0,17883.0,19872.0,0.96,9.33,9.48,19.69,78.0,0.90,1.45,16.76,XY25,NTT,AC
35,ICICIGI,2252.93,-15.31,56.0,X-MC,7.54,188399.0,14079.0,21120.0,-0.84,8.08,11.21,20.19,68.0,0.67,1.30,24.41,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INFY,2275.00,-18.52,59.0,X-LC,3.62,327637.0,14749.0,156938.0,3.49,4.71,47.90,54.87,2.0,0.09,2.27,13.39,X40,BTT,IT
13,BSOFT,831.70,-8.77,61.0,H-SC,8.21,100011.0,-40642.0,112072.0,3.25,-28.90,112.06,50.79,151.0,-0.36,0.69,16.45,XR,ATH,IT
32,HCLTECH,1922.01,11.49,75.0,X-LC,4.35,258061.0,16137.0,43690.0,3.04,6.67,16.93,24.73,7.0,0.37,1.79,22.92,X40,ATH,IT
50,LTIM,7201.88,3.99,65.0,H-LC,11.41,265928.0,19777.0,58158.0,2.67,8.03,21.87,31.66,43.0,0.34,1.84,47.55,X200,ATH,IT
86,WIPRO,420.00,-13.79,54.0,M-LC,5.14,151950.0,1005.0,108447.0,1.74,0.67,71.37,72.51,99.0,0.01,1.05,6.73,XR,NTT,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,103.12,58.0,H-SC,16.06,138174.0,-80322.0,108508.0,-4.18,-36.76,78.53,12.90,161.0,-0.74,0.96,14.89,OX40N,NTT,FINANCE
51,MASFIN,398.61,-18.25,51.0,H-SC,7.17,92760.0,-5220.0,26826.0,-2.55,-5.33,28.92,22.05,164.0,-0.19,0.64,34.83,XR,ATH,FINANCE
63,SAMMAANCAP,326.00,18.70,53.0,M-SC,36.59,160659.0,11439.0,132736.0,-2.33,7.67,82.62,96.62,245.0,0.09,1.11,73.87,XY25,NTT,FINANCE
18,COFFEEDAY,80.00,-53.72,31.0,L-SC,19.99,69370.0,-44179.0,84229.0,-2.25,-38.91,121.42,35.27,270.0,-0.52,0.48,69.78,XR,NTT,HOTELS
58,RAJOOENG,143.10,-48.54,28.0,H-SC,18.20,78050.0,-24450.0,65047.0,-2.18,-23.85,83.34,39.61,135.0,-0.38,0.54,0.00,AR,ATH,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
70,STARHEALTH,761.0,35.33,64.0,H-SC,15.58,278936.0,-2187.0,135061.0,-0.54,-0.78,48.42,47.26,171.0,-0.02,1.93,49.45,XY24,NTT,INSURANCE
86,WIPRO,420.0,-13.79,54.0,M-LC,5.14,151950.0,1005.0,108447.0,1.74,0.67,71.37,72.51,99.0,0.01,1.05,6.73,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-20.55,41.0,M-SC,10.14,144364.0,-3280.0,162612.0,-1.67,-2.22,112.64,107.91,216.0,-0.02,1.00,0.71,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,146.65,79.0,M-SC,1.89,161777.0,-12782.0,12813.0,-1.61,-7.32,7.92,0.02,234.0,-1.00,1.12,40.72,OX40N,NTT,PAINTS
79,TTKPRESTIG,770.00,100.85,50.0,M-SC,3.72,86526.0,-14251.0,14346.0,0.09,-14.14,16.58,0.09,253.0,-0.99,0.60,13.68,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-66.37,43.0,H-SC,1.99,220500.0,-49167.0,85510.0,-0.80,-18.23,38.78,13.47,158.0,-0.57,1.53,12.12,XY24,NTT,PAINTS
44,JCHAC,2282.00,19986.96,65.0,M-SC,1.44,98140.0,-29665.0,29648.0,0.01,-23.21,30.21,-0.01,233.0,-1.00,0.68,11.13,OX40N,NTT,AC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SAMMAANCAP,326.00,18.70,53.0,M-SC,36.59,160659.0,11439.0,132736.0,-2.33,7.67,82.62,96.62,245.0,0.09,1.11,73.87,XY25,NTT,FINANCE
86,WIPRO,420.00,-13.79,54.0,M-LC,5.14,151950.0,1005.0,108447.0,1.74,0.67,71.37,72.51,99.0,0.01,1.05,6.73,XR,NTT,IT
38,INDIAMART,4810.62,-48.72,44.0,H-SC,4.56,127730.0,4394.0,127232.0,-1.48,3.56,99.61,106.72,139.0,0.03,0.88,27.77,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-48.72,44.0,H-SC,4.56,127730.0,4394.0,127232.0,-1.48,3.56,99.61,106.72,139.0,0.03,0.88,27.77,AR,ATH,MISC
86,WIPRO,420.00,-13.79,54.0,M-LC,5.14,151950.0,1005.0,108447.0,1.74,0.67,71.37,72.51,99.0,0.01,1.05,6.73,XR,NTT,IT
25,FINCABLES,1641.55,-20.55,41.0,M-SC,10.14,144364.0,-3280.0,162612.0,-1.67,-2.22,112.64,107.91,216.0,-0.02,1.00,0.71,OX40N,ATH,CABLES
85,WHIRLPOOL,2270.00,-43.00,36.0,M-SC,7.86,111642.0,-3156.0,97196.0,0.55,-2.75,87.06,81.92,236.0,-0.03,0.77,32.17,XR,NTT,DURABLES
87,ZYDUSLIFE,1286.17,-15.85,33.0,H-MC,5.23,205254.0,-4320.0,78982.0,0.04,-2.06,38.48,35.63,119.0,-0.05,1.42,14.27,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TMPV,1065.00,-23.08,22.0,X-LC,0.26,238682.0,-37728.0,458890.0,-1.86,-13.65,192.26,152.37,3.0,-0.08,1.65,0.00,XY24,NTT,AUTO
11,BATAINDIA,2096.00,10.04,24.0,X-SC,10.90,83896.0,-44774.0,90071.0,-0.39,-34.80,107.36,35.20,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
34,HONAUT,58357.33,-26.58,28.0,X-SC,1.31,104370.0,-23568.0,70700.0,-0.11,-18.42,67.74,36.84,90.0,-0.33,0.72,6.85,X40N,ATH,ELECTRICAL
26,GILLETTE,11206.78,-19.05,29.0,X-SC,6.10,250155.0,-3741.0,86053.0,-0.06,-1.47,34.40,32.42,85.0,-0.04,1.73,13.40,X40,ATH,FMCG
56,QUESS,424.00,-54.27,30.0,X-SC,37.29,45620.0,-19386.0,47235.0,-1.15,-29.82,103.54,42.84,83.0,-0.41,0.32,0.00,XY24,NTT,MISC


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TMPV,1065.00,-23.08,22.0,X-LC,0.26,238682.0,-37728.0,458890.0,-1.86,-13.65,192.26,152.37,3.0,-0.08,1.65,0.00,XY24,NTT,AUTO
43,ITC,452.00,-41.15,41.0,X-LC,0.92,197043.0,-3095.0,23980.0,-0.71,-1.55,12.17,10.44,5.0,-0.13,1.36,3.93,X40,NTT,FMCG
34,HONAUT,58357.33,-26.58,28.0,X-SC,1.31,104370.0,-23568.0,70700.0,-0.11,-18.42,67.74,36.84,90.0,-0.33,0.72,6.85,X40N,ATH,ELECTRICAL
36,ICICIPRULI,790.00,-21.49,56.0,X-MC,1.45,181411.0,4673.0,50051.0,-1.22,2.64,27.59,30.97,75.0,0.09,1.26,15.54,X40,ATH,INSURANCE
67,SIEMENS,4671.50,-1.82,58.0,H-LC,1.76,159990.0,-26105.0,73579.0,-1.53,-14.03,45.99,25.51,49.0,-0.35,1.11,17.90,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,10.04,24.0,X-SC,10.90,83896.0,-44774.0,90071.0,-0.39,-34.80,107.36,35.20,92.0,-0.50,0.58,0.0,X40,NTT,FOOTWEAR
56,QUESS,424.00,-54.27,30.0,X-SC,37.29,45620.0,-19386.0,47235.0,-1.15,-29.82,103.54,42.84,83.0,-0.41,0.32,0.0,XY24,NTT,MISC
54,PAGEIND,51605.07,-30.18,33.0,X-MC,8.19,155200.0,-8540.0,51216.0,-0.84,-5.22,33.00,26.07,55.0,-0.17,1.07,0.0,X40,ATH,APPARELS
77,TMPV,1065.00,-23.08,22.0,X-LC,0.26,238682.0,-37728.0,458890.0,-1.86,-13.65,192.26,152.37,3.0,-0.08,1.65,0.0,XY24,NTT,AUTO
9,BAJAJHFL,181.50,-19.44,30.0,X-MC,6.93,173938.0,-26568.0,123722.0,-0.49,-13.25,71.13,48.45,64.0,-0.21,1.20,0.3,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,-54.27,30.0,X-SC,37.29,45620.0,-19386.0,47235.0,-1.15,-29.82,103.54,42.84,83.0,-0.41,0.32,0.00,XY24,NTT,MISC
60,RELAXO,1176.00,-42.41,34.0,X-SC,3.11,78509.0,-66651.0,142580.0,-0.90,-45.92,181.61,52.31,91.0,-0.47,0.54,5.09,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,10.04,24.0,X-SC,10.90,83896.0,-44774.0,90071.0,-0.39,-34.80,107.36,35.20,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-14.73,50.0,X-MC,3.84,89010.0,-1668.0,16574.0,0.02,-1.84,18.62,16.44,63.0,-0.10,0.62,17.96,X40,ATH,PHARMA
34,HONAUT,58357.33,-26.58,28.0,X-SC,1.31,104370.0,-23568.0,70700.0,-0.11,-18.42,67.74,36.84,90.0,-0.33,0.72,6.85,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.97,-25.57,63.0,X-LC,8.12,295216.0,-50740.0,117437.0,1.73,-14.67,39.78,19.28,1.0,-0.43,2.04,9.14,X40,ATH,IT
41,INFY,2275.00,-18.52,59.0,X-LC,3.62,327637.0,14749.0,156938.0,3.49,4.71,47.90,54.87,2.0,0.09,2.27,13.39,X40,BTT,IT
77,TMPV,1065.00,-23.08,22.0,X-LC,0.26,238682.0,-37728.0,458890.0,-1.86,-13.65,192.26,152.37,3.0,-0.08,1.65,0.00,XY24,NTT,AUTO
83,VBL,671.64,-19.52,43.0,X-LC,5.93,288408.0,-27434.0,140109.0,-1.28,-8.69,48.58,35.67,4.0,-0.20,2.00,3.96,X40N,ATH,FMCG
43,ITC,452.00,-41.15,41.0,X-LC,0.92,197043.0,-3095.0,23980.0,-0.71,-1.55,12.17,10.44,5.0,-0.13,1.36,3.93,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,7577.78,61.0,L-SC,18.01,84219.0,-9591.0,86207.0,-1.83,-10.22,102.36,81.67,271.0,-0.11,0.58,63.09,XR,NTT,CERAMICS
51,MASFIN,398.61,-18.25,51.0,H-SC,7.17,92760.0,-5220.0,26826.0,-2.55,-5.33,28.92,22.05,164.0,-0.19,0.64,34.83,XR,ATH,FINANCE
71,SURYODAY,216.00,53.51,44.0,H-SC,12.63,141148.0,-37923.0,79170.0,0.18,-21.18,56.09,23.03,167.0,-0.48,0.98,40.05,XR,NTT,BANKS
81,VAIBHAVGBL,521.00,63.09,49.0,H-SC,10.75,141620.0,-41155.0,152227.0,-2.03,-22.52,107.49,60.77,159.0,-0.27,0.98,27.54,XR,NTT,JEWELLERY
66,SHALBY,327.00,1043.54,37.0,M-SC,16.92,155454.0,-25689.0,71151.0,0.19,-14.18,45.77,25.10,250.0,-0.36,1.08,24.58,XY24,NTT,HEALTHCARE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,146.65,79.0,M-SC,1.89,161777.0,-12782.0,12813.0,-1.61,-7.32,7.92,0.02,234.0,-1.00,1.12,40.72,OX40N,NTT,PAINTS
13,BSOFT,831.70,-8.77,61.0,H-SC,8.21,100011.0,-40642.0,112072.0,3.25,-28.90,112.06,50.79,151.0,-0.36,0.69,16.45,XR,ATH,IT
5,ASIANPAINT,3460.25,0.49,83.0,X-LC,10.87,255561.0,3793.0,48940.0,-0.07,1.51,19.15,20.95,31.0,0.08,1.77,37.68,X40,ATH,PAINTS
81,VAIBHAVGBL,521.00,63.09,49.0,H-SC,10.75,141620.0,-41155.0,152227.0,-2.03,-22.52,107.49,60.77,159.0,-0.27,0.98,27.54,XR,NTT,JEWELLERY
40,INDUSINDBK,1800.00,-690.94,64.0,L-MC,5.72,47877.0,-29330.0,54723.0,-0.94,-37.99,114.30,32.89,259.0,-0.54,0.33,31.85,XR,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.63
1,25,44.49
2,50,75.19


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    46.55
MC    28.68
LC    24.79
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.93
X40      20.63
X40N     11.50
XY25     10.13
XR        9.48
AR        9.02
OX40N     7.67
X200      1.84
SR        1.01
MH        0.81
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.22
X-MC    22.30
X-LC    19.82
M-SC    13.22
X-SC     7.71
H-MC     4.69
H-LC     2.95
L-SC     1.40
M-MC     1.36
M-LC     1.05
L-LC     0.97
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.62,-4.87,38.78
IT,13.58,-13.51,72.16
FINANCE,10.46,-12.32,65.14
MISC,7.37,-21.93,71.88
PAINTS,6.07,-5.40,21.05
ELECTRICAL,5.62,-11.28,50.73
INSURANCE,4.49,2.55,31.79
PHARMA,3.42,-1.12,35.04
AUTO,3.32,-19.51,134.09


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3284635.0,21
AR,1266347.0,10
XR,1260924.0,13
X40,987370.0,14
X40N,709097.0,9
OX40N,684985.0,10
XY25,469462.0,7
SR,275566.0,2
MH,67471.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3478496.0,25
M-SC,1441712.0,16
X-LC,1204861.0,11
X-MC,1171196.0,16
X-SC,725813.0,8
H-MC,401673.0,3
L-SC,258214.0,3
H-LC,131737.0,2
M-LC,108447.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1182106.0      6
           AR         851137.0      5
           XR         781781.0      7
M-SC       XY24       762156.0      6
X-LC       XY24       530101.0      2
X-MC       X40        420261.0      7
X-LC       X40        390985.0      5
X-MC       XY24       344542.0      3
H-SC       OX40N      320435.0      4
X-SC       XY24       281177.0      3
M-SC       OX40N      276772.0      5
H-SC       SR         275566.0      2
X-SC       X40N       268512.0      3
X-MC       X40N       231225.0      4
H-MC       AR         217120.0      2
X-LC       X40N       209360.0      2
H-MC       XY24       184553.0      1
X-SC       X40        176124.0      2
X-MC       XY25       175168.0      2
L-SC       XR         170436.0      2
M-SC       XR         145537.0      2
           XY25       132736.0      1
           AR         124511.0      2
M-LC       XR         108447.0      1
L-SC       OX40N       87778.0      1
X-LC       XY25        74415.0      2
H-LC       AR          73579.0      1
H-SC       MH          67471.0      1
H-LC       X200        58158.0      1
L-MC       XR          54723.0      1
M-MC       XY25        48398.0      1
L-LC       XY25        38745.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 51.0 seconds
